In [15]:
"""
Machine learning project for me to try out shape detection
"""
from pathlib import Path
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, dataloader

print(f'cwd: {Path.cwd()}')

def split_folder(folder):
    all_files = Path(f'{folder}/').glob(f'*.png')
    all_files = list(all_files)
    print(list(all_files)[0:3])
    random.shuffle(all_files)
    split = 0.7
    split_index = int(np.floor(len(all_files) * split))
    training = all_files[0:split_index]
    testing = all_files[split_index:len(all_files)]
    return training, testing

Path.mkdir(Path('./train'), exist_ok=True)
Path.mkdir(Path('./validate'), exist_ok=True)

# data = {}
# data[shape] = split_folder(shape)
for shape in ['circle', 'star', 'triangle', 'square']:
    print(f'shape: {shape}')
    for dataset in ['train', 'validate']:
        Path.mkdir(Path(f'./{dataset}/{shape}'), exist_ok=True)
    train, validate = split_folder(shape)
    for file in train:
        shutil.copy(file, f'./train/{shape}')
    for file in validate:
        shutil.copy(file, f'./validate/{shape}')
    


cwd: c:\Users\sylvi\Documents\GitKraken\python-tests\ML\Shape Detection
shape: circle
[WindowsPath('circle/0.png'), WindowsPath('circle/1.png'), WindowsPath('circle/10.png')]
shape: star
[WindowsPath('star/0.png'), WindowsPath('star/1.png'), WindowsPath('star/10.png')]
shape: triangle
[WindowsPath('triangle/0.png'), WindowsPath('triangle/1.png'), WindowsPath('triangle/10.png')]
shape: square
[WindowsPath('square/0.png'), WindowsPath('square/1.png'), WindowsPath('square/10.png')]


In [16]:
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True
    print('cuda detected!')

simple_transform = transforms.Compose([
    transforms.Resize((64, 64)), 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

train = ImageFolder('./train', simple_transform)
valid = ImageFolder('./validate', simple_transform)

train_data_loader = torch.utils.data.DataLoader(train, batch_size=32, num_workers=3, shuffle=True)
valid_data_loader = torch.utils.data.DataLoader(valid, batch_size=32, num_workers=3, shuffle=True)



cuda detected!


In [18]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(3380, 50)
        self.fc2 = nn.Linear(50, 4)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x, dim=1)

In [21]:
def fit(epoch, model, data_loader, phase='training', volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile = True
    running_loss = 0.0
    running_correct = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        if is_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile), Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)

        #running_loss += F.nll_loss(output, target, size_average=False).data[0]
        running_loss += F.nll_loss(output, target, size_average=False).data
        preds = output.data.max(dim=1, keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()

    loss = running_loss / len(data_loader.dataset)
    accuracy = 100. * running_correct / len(data_loader.dataset)

    print(f'{phase} loss is {loss} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)} = {accuracy}')
    return loss, accuracy

In [22]:
model = Net()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.5)
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]
for epoch in range(1,20):
    epoch_loss, epoch_accuracy = fit(epoch,model,train_data_loader,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,valid_data_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)




RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same